In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from math import sqrt
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier 
from sklearn.model_selection import cross_val_score,cross_val_predict,train_test_split,GridSearchCV,KFold,StratifiedKFold

In [2]:
data = pd.read_csv('../gtmsa_practicum_datasets/merged_final_data.csv',dtype=str)
# rearrange column (put reponse colkumn to the end)
data = data[[c for c in data if c not in ['relative price for inpatient and outpatient services']] 
       + ['relative price for inpatient and outpatient services']]
# remove % at the end of these 2 columns
data['relative price for inpatient facility services'] = data['relative price for inpatient facility services'].str.rstrip('%').astype('float')
data['relative price for outpatient facility services'] = data['relative price for outpatient facility services'].str.rstrip('%').astype('float')
# convert all to numeric
data = data.apply(pd.to_numeric, errors='ignore')
# convert object data type column to dummy variables
labelencoder = LabelEncoder()
cat_features=[x for x in data.columns if data[x].dtype=="object"]
for col in cat_features:
    if col in data.columns:
        i = data.columns.get_loc(col)
        data.iloc[:,i] = data.apply(lambda i:labelencoder.fit_transform(i.astype(str)), axis=0, result_type='expand')

print(data.shape)


(3329, 87)


In [3]:
# filter to only labeled data
idx = data['relative price for inpatient and outpatient services'].isnull()
dt = data[~idx].reset_index(drop=True)
# fill NaN with imputation
dt.fillna(method="ffill", inplace=True)
dt.fillna(method="bfill", inplace=True)
print(dt.shape)
dt.head()

(1628, 87)


,medicare provider number,hospital name,street address,city,state,zip code,"hospital system or, if independent, ipps/cah",is hospital a critical access hospital (y/n)?,"hospital compare 5-star rating (october 2018, na=not available)",number of outpatient services,...,Revenue from rental of living quarters [other_income_livingrental],Revenue from laundry and linen service [other_income_laundry],Parking lot receipts [other_income_parking],Rebates and refunds of expenses [other_income_rebates],Nursing and allied health managed care payment [mdcr_ipps_nurs_mgdcare_pymts],Net organ acquisition cost [mdcr_ipps_net_organ_cost],Cost of teaching physicians [mdcr_ipps_teach_phys_cost],Routine service other pass through costs [mdcr_ipps_routine_passthru_cost],medicare provider number.1,relative price for inpatient and outpatient services
0,10006,732,1231,587,1,35631,289,0,3.0,118.0,...,759030.638055,47940.640042,538219.675038,144220.931231,161604.634219,7.292181e+06,52477.976199,293090.969362,100002.0,241.0
1,10029,704,1177,1330,1,36801,152,0,4.0,603.0,...,759030.638055,47940.640042,538219.675038,144220.931231,161604.634219,7.292181e+06,52477.976199,293090.969362,100002.0,174.0
2,10033,3003,2643,152,1,35233,540,0,2.0,345.0,...,759030.638055,47940.640042,538219.675038,144220.931231,161604.634219,7.292181e+06,52477.976199,293090.969362,100002.0,322.0
3,10039,1124,163,817,1,35801,244,0,2.0,1091.0,...,759030.638055,47940.640042,538219.675038,144220.931231,161604.634219,7.292181e+06,52477.976199,293090.969362,100002.0,245.0
4,10056,2587,3003,152,1,35205,23,0,4.0,107.0,...,759030.638055,47940.640042,538219.675038,144220.931231,161604.634219,7.292181e+06,52477.976199,293090.969362,100002.0,174.0


In [4]:
# split dataset to x and y
random_state = 100
x_data = dt.loc[:, dt.columns != "relative price for inpatient and outpatient services"]
y_data = dt.loc[:, "relative price for inpatient and outpatient services"]

# Linear Regression

In [5]:
#split dataset
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, shuffle=True, random_state=random_state)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1221, 86), (407, 86), (1221,), (407,))

In [6]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import BayesianRidge, Ridge, ElasticNet, LinearRegression


model_factory = [
 RandomForestRegressor(),
 LinearRegression(),
 Ridge(),
 BayesianRidge(),
 ExtraTreesRegressor(),
 ElasticNet(),
 KNeighborsRegressor(),
 GradientBoostingRegressor()
]

for model in model_factory:
    model.seed = 222
    num_folds = 5

    y_pred = cross_val_predict(model, x_train, y_train, cv=num_folds)
    score_description = " %0.2f (+/- %0.2f)" % (np.sqrt(mean_squared_error(y_train, y_pred)), y_pred.std() * 2)
    
    print('{model:25} CV-5 RMSE: {score}'.format(
     model=model.__class__.__name__,
     score=score_description
     ))
    

RandomForestRegressor     CV-5 RMSE:  19.51 (+/- 142.53)
LinearRegression          CV-5 RMSE:  40.58 (+/- 167.42)
Ridge                     CV-5 RMSE:  37.06 (+/- 161.78)
BayesianRidge             CV-5 RMSE:  35.31 (+/- 156.68)
ExtraTreesRegressor       CV-5 RMSE:  20.81 (+/- 141.58)
ElasticNet                CV-5 RMSE:  35.85 (+/- 158.29)
KNeighborsRegressor       CV-5 RMSE:  75.56 (+/- 86.44)
GradientBoostingRegressor CV-5 RMSE:  17.66 (+/- 144.46)


In [7]:
import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics


classification_model_factory = [
 LogisticRegression(),
 SGDClassifier(),
 SVC(),
 KNeighborsClassifier(),
 GaussianNB(),
 DecisionTreeClassifier(),
 RandomForestClassifier(),
 AdaBoostClassifier(),
 GradientBoostingClassifier(),
 MLPClassifier()
]

for model in classification_model_factory:
    model.seed = 222
    num_folds = 5

    y_pred = cross_val_predict(model, x_train, y_train, cv=num_folds)
    accuracy = metrics.accuracy_score(y_train, y_pred)
    precision = metrics.precision_score(y_train, y_pred, average='weighted')
    f1_score = metrics.precision_score(y_train,y_pred, average='weighted')
    score_description = "Accuracy - %0.2f , precision -  %0.2f , f1_score - %0.2f" % (accuracy, precision,f1_score)
    
    print('{model:25} CV-5 Performance: {score}'.format(
     model=model.__class__.__name__,
     score=score_description
     ))

LogisticRegression        CV-5 Performance: Accuracy - 0.01 , precision -  0.00 , f1_score - 0.00
SGDClassifier             CV-5 Performance: Accuracy - 0.00 , precision -  0.00 , f1_score - 0.00
SVC                       CV-5 Performance: Accuracy - 0.01 , precision -  0.00 , f1_score - 0.00
KNeighborsClassifier      CV-5 Performance: Accuracy - 0.00 , precision -  0.00 , f1_score - 0.00
GaussianNB                CV-5 Performance: Accuracy - 0.01 , precision -  0.01 , f1_score - 0.01
DecisionTreeClassifier    CV-5 Performance: Accuracy - 0.02 , precision -  0.02 , f1_score - 0.02
RandomForestClassifier    CV-5 Performance: Accuracy - 0.01 , precision -  0.01 , f1_score - 0.01
AdaBoostClassifier        CV-5 Performance: Accuracy - 0.01 , precision -  0.00 , f1_score - 0.00
GradientBoostingClassifier CV-5 Performance: Accuracy - 0.01 , precision -  0.01 , f1_score - 0.01
MLPClassifier             CV-5 Performance: Accuracy - 0.01 , precision -  0.00 , f1_score - 0.00
